<a href="https://colab.research.google.com/github/diegocgribeiro/previsao_brasileirao24/blob/main/modeloregreessaobr24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas sqlalchemy pymysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
pip install mysql-connector-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 40.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import mysql.connector

conexao = mysql.connector.connect(
    host='datasite.mysql.uhserver.com',
    user='data_user',
    password='r@otPass2003',
    database='datasite'
)
query = "SELECT * FROM tabela_campeonato"
df = pd.read_sql(query, conexao)
conexao.close()


<ipython-input-49-35bb8d990d7f>:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conexao)


In [ ]:
def classificacao(df):
  classd = df.sort_values(by=['pontos', 'vitorias', 'sg', 'gp'], ascending=False)
  return classd

In [ ]:
def enviar_para_mysql(df, tabela):
    try:
        conexao = mysql.connector.connect(
        host='datasite.mysql.uhserver.com',
        user='data_user',
        password='r@otPass2003',
        database='datasite'
)
        cursor = conexao.cursor()
        cursor.execute(f"""
        CREATE TABLE IF NOT EXISTS {tabela} (
            Posição INT,
            Times VARCHAR(100),
            Pontos FLOAT,
            Jogos INT,
            Jogos_Casa INT,
            Jogos_Fora INT,
            Vitorias INT,
            Vitorias_Casa INT,
            Vitorias_Fora INT,
            Derrotas INT,
            Derrotas_Casa INT,
            Derrotas_Fora INT,
            Empates INT,
            Empates_Casa INT,
            Empates_Fora INT,
            Gols_Feitos INT,
            Gols_Feitos_Casa INT,
            Gols_Feitos_Fora INT,
            Gols_Tomados INT,
            Gols_Tomados_Casa INT,
            Gols_Tomados_Fora INT,
            Saldo_de_Gols INT,
            Saldo_de_Gols_Casa INT,
            Saldo_de_Gols_Fora INT,
            Aproveitamento FLOAT,
            Aproveitamento_Casa FLOAT,
            Aproveitamento_Fora FLOAT,
            Rodada INT
        )
        """)

        # Insert data from classificacoes_df into the table
        for index, row in classificacoes_df.iterrows():
            cursor.execute(f"""
            INSERT INTO {tabela} (Posição, Times, Pontos, Jogos, Jogos_Casa, Jogos_Fora, Vitorias, Vitorias_Casa, Vitorias_Fora,
                                  Derrotas, Derrotas_Casa, Derrotas_Fora, Empates, Empates_Casa, Empates_Fora,
                                  Gols_Feitos, Gols_Feitos_Casa, Gols_Feitos_Fora, Gols_Tomados, Gols_Tomados_Casa,
                                  Gols_Tomados_Fora, Saldo_de_Gols, Saldo_de_Gols_Casa, Saldo_de_Gols_Fora,
                                  Aproveitamento, Aproveitamento_Casa, Aproveitamento_Fora, Rodada)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, tuple(row))

        # Commit changes
        conexao.commit()

        print(f'Dados inseridos com sucesso na tabela {tabela}')

    except mysql.connector.Error as err:
        print({err})

    finally:
        if conexao.is_connected():
            cursor.close()
            conexao.close()

**PREVISÃO! APÓS TODOS OS DADOS ESTAREM CORRETO**

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
for i in range(30,39):
  df_filtered = df[(df['rodada'] <= 29) & (df['rodada'] >= 0)]
  previsoes_lista = []
  times = df['time'].unique()
  for time in times:
      df_time = df_filtered[df_filtered['time'] == time]
      previsoes_time = {'time': time, 'jogos': i}
      for var in ['vitorias', 'empates', 'derrotas', 'gp', 'gc', 'sg']:
          y = df_time[var]
          x= df_time[['jogos']]
          model = LinearRegression()
          x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
          model.fit(x_train, y_train)
          jogo = pd.DataFrame({'jogos': [i]})
          previsao_var = model.predict(jogo)
          previsoes_time[var] = previsao_var[0]
      previsoes_time['pontos'] = (previsoes_time['vitorias'].round(0) * 3) + (previsoes_time['empates'].round(0) * 1)
      previsoes_lista.append(previsoes_time)
  df_pred_rodada = pd.DataFrame(previsoes_lista)
  df_pred_rodada = df_pred_rodada.round(0)
  df_pred_rodada = classificacao(df_pred_rodada)
  df_pred_rodada['Posição'] = range(1, len(df_pred_rodada) + 1)
  enviar_para_mysql(df_pred_rodada, 'previsoes_atualizadacampinteiro')

Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro
Dados inseridos com sucesso na tabela previsoes_atualizadacampinteiro


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
times = [
    "Internacional", "Bahia", "Criciúma", "Juventude", "Fluminense",
    "Red Bull Bragantino", "São Paulo", "Fortaleza", "Atlético-GO",
    "Flamengo", "Vasco", "Grêmio", "Corinthians", "Atlético-MG",
    "Athletico-PR", "Cuiabá", "Cruzeiro", "Botafogo", "Vitória", "Palmeiras", "Inter", "Bragantino"
]

url = "https://olympics.com/pt/noticias/campeonato-brasileiro-2024-todos-resultados"

# Adicionar um cabeçalho User-Agent para simular um navegador real
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    results = soup.find_all('li')
    data = []
    rodadas = []
    for result in results:
        result_text = result.get_text().strip()
        if any(time in result_text for time in times):
            data.append({"Resultado": result_text})
else:
    print(f"Erro: Não foi possível acessar a página. Status Code: {response.status_code}")

df = pd.DataFrame(data)

In [ ]:
for i in range(1, 39):
    for j in range(10):
      rodadas.append(i)

df['Rodada'] = rodadas
df = df[~df['Resultado'].str.contains(r'A definir|adiado', case=False, na=False)]
df['Resultado'] = df['Resultado'].replace({'Atlético-MG': 'AtléticoMG', 'Atlético-GO': 'AtléticoGO', 'Athletico-PR': 'AthleticoPR', 'São Paulo': 'SPFC', 'Red Bull Bragantino': 'Bragantino', 'Internacional': 'Inter'}, regex=True)
df[['Time A']] = df['Resultado'].str.extract(r'(\w+)')
df[['Time B']] = df['Resultado'].str.extract(r'(\w+)$')
df[['Gols Time A']] = df['Resultado'].str.extract(r'(\d+)')
df[['Gols Time B']] = df['Resultado'].str.extract(r'x (\d+)')
times_a = set(df['Time A'].unique())
times_b = set(df['Time B'].unique())

df.to_csv('resultados.csv', index=False)

<ipython-input-2-d7e534e04bbb>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Resultado'] = df['Resultado'].replace({'Atlético-MG': 'AtléticoMG', 'Atlético-GO': 'AtléticoGO', 'Athletico-PR': 'AthleticoPR', 'São Paulo': 'SPFC', 'Red Bull Bragantino': 'Bragantino', 'Internacional': 'Inter'}, regex=True)
<ipython-input-2-d7e534e04bbb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['Time A']] = df['Resultado'].str.extract(r'(\w+)')
<ipython-input-2-d7e534e04bbb>:9: SettingWithCopyWarning: 
A value i

In [ ]:
import pandas as pd
df = pd.read_csv('resultados.csv')
futuros_jogos = df[df['Gols Time A'].isna()]
df = df.dropna(subset=['Gols Time A'])
classificacoes_por_rodada = []
for rodada in range(1, df['Rodada'].max()+1):
  df_rodada = df[df['Rodada'] <= rodada]
  classificacao = pd.DataFrame()
  classificacao['Posição'] = 0
  classificacao['Times'] = df['Time A'].unique()
  classificacao['Pontos'] = 0
  classificacao['Jogos'] = 0
  classificacao['Jogos Casa'] = 0
  classificacao['Jogos Fora'] = 0
  classificacao['Vitorias'] = 0
  classificacao['Vitorias Casa'] = 0
  classificacao['Vitorias Fora'] = 0
  classificacao['Derrotas'] = 0
  classificacao['Derrotas Casa'] = 0
  classificacao['Derrotas Fora'] = 0
  classificacao['Empates'] = 0
  classificacao['Empates Casa'] = 0
  classificacao['Empates Fora'] = 0
  classificacao['Gols Feitos'] = 0
  classificacao['Gols Feitos Casa'] = 0
  classificacao['Gols Feitos Fora'] = 0
  classificacao['Gols Tomados'] = 0
  classificacao['Gols Tomados Casa'] = 0
  classificacao['Gols Tomados Fora'] = 0
  classificacao['Saldo de Gols'] = 0
  classificacao['Saldo de Gols Casa'] = 0
  classificacao['Saldo de Gols Fora'] = 0
  classificacao['Aproveitamento'] = 0
  classificacao['Aproveitamento Casa'] = 0
  classificacao['Aproveitamento Fora'] = 0
  for index, row in df_rodada.iterrows():
    if row['Gols Time A'] > row['Gols Time B']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Vitorias Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Derrotas Fora'] += 1
    elif row['Gols Time B'] > row['Gols Time A']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Derrotas Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Vitorias Fora'] += 1
    else:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Empates Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Empates Fora'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Feitos Casa'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Feitos Fora'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Tomados Casa'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Tomados Fora'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Jogos Casa'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Jogos Fora'] += 1
  principais =['Jogos','Vitorias', 'Derrotas', 'Empates','Jogos', 'Gols Feitos', 'Gols Tomados']
  for i in principais:
    classificacao[i] = classificacao[f'{i} Casa'] + classificacao[f'{i} Fora']
  #classificacao['Jogos'] = classificacao['Jogos Casa'] + classificacao['Jogos Fora']
  #classificacao['Vitorias'] = classificacao['Vitorias Casa'] + classificacao['Vitorias Fora']
  classificacao['Saldo de Gols Casa'] = classificacao['Gols Feitos Casa'] - classificacao['Gols Tomados Casa']
  classificacao['Saldo de Gols Fora'] = classificacao['Gols Feitos Fora'] - classificacao['Gols Tomados Fora']
  classificacao['Pontos'] = classificacao['Vitorias'] * 3 + classificacao['Empates'] * 1
  classificacao['Aproveitamento'] = classificacao['Pontos']/(classificacao['Jogos']*3)
  classificacao['Aproveitamento Casa'] = (classificacao['Vitorias Casa']*3 +classificacao['Empates Casa']*1)/(classificacao['Jogos Casa']*3)
  classificacao['Aproveitamento Fora'] = (classificacao['Vitorias Fora']*3 +classificacao['Empates Fora']*1)/(classificacao['Jogos Fora']*3)
  classificacao['Saldo de Gols'] = classificacao['Gols Feitos'] - classificacao['Gols Tomados']
  classificacao = classificacao.sort_values(by=['Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
  classificacao['Posição'] = list(range(1, 21))
  classificacao['Rodada'] = rodada
  classificacoes_por_rodada.append(classificacao)
classificacoes_df = pd.concat(classificacoes_por_rodada)
classificacoes_df = classificacoes_df.sort_values(by=['Rodada', 'Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
classificacoes_df.to_excel('tabela_brasileiraoate30.xlsx')
minhabase = classificacoes_df

In [ ]:
enviar_para_mysql(minhabase, 'tabela_minhabase2')

Dados inseridos com sucesso na tabela tabela_minhabase2


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('resultados.csv')
futuros_jogos = df[df['Gols Time A'].isna()]

grande = [
    "Palmeiras", "Flamengo", "SPFC", "Inter",
    "Cruzeiro", "Grêmio", "Fluminense", "Corinthians"
]

medio = [
    "Botafogo", "Fortaleza", "Bahia",
    "AtléticoMG", "Vasco", "AthleticoPR"
]

pequeno = [
    "Criciúma", "Bragantino", "Juventude",
    "Vitória", "Cuiabá", "AtléticoGO"
]

for index, row in futuros_jogos.iterrows():

  gols_feitos_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Feitos Casa'].values[0]
  jogos_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Casa'].values[0]
  gols_tomados_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Tomados Fora'].values[0]
  jogos_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Fora'].values[0]
  posicao_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Posição'].values[0]
  posicao_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Posição'].values[0]
  pontos5jogosatras = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()-5), 'Pontos'].values[0]
  pontosagora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Pontos'].values[0]
  if row['Time A'] in grande:
    fator_grandeza = 1.1
  elif row['Time A'] in medio:
    fator_grandeza = 1
  else:
    fator_grandeza = 0.9
  media_gols_casa_feitos = gols_feitos_casa/jogos_casa
  media_gols_fora_tomados = gols_tomados_fora/jogos_fora
  fator5jogos = 1+((pontosagora - pontos5jogosatras)/50)
  futuros_jogos.loc[(futuros_jogos['Time A'] == row['Time A']) & (futuros_jogos['Time B'] == row['Time B']), 'Gols Time A'] = ((((media_gols_casa_feitos+media_gols_fora_tomados)/2)*(1+(((20-posicao_casa)*2))/100)*fator_grandeza)*fator5jogos).round(0)

  gols_feitos_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Feitos Fora'].values[0]
  jogos_fora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Fora'].values[0]
  gols_tomados_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Gols Tomados Casa'].values[0]
  jogos_casa = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time A']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Jogos Casa'].values[0]
  pontos5jogosatras = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()-5), 'Pontos'].values[0]
  pontosagora = classificacoes_df.loc[(classificacoes_df['Times'] == row['Time B']) & (classificacoes_df['Rodada'] == classificacoes_df['Rodada'].max()), 'Pontos'].values[0]
  if row['Time B'] in grande:
    fator_grandeza = 1.1
  elif row['Time B'] in medio:
    fator_grandeza = 1
  else:
    fator_grandeza = 0.9
  fator5jogos = 1+((pontosagora - pontos5jogosatras)/50)
  media_gols_fora_feitos = gols_feitos_fora/jogos_fora
  media_gols_casa_tomados = gols_tomados_casa/jogos_casa
  futuros_jogos.loc[(futuros_jogos['Time A'] == row['Time A']) & (futuros_jogos['Time B'] == row['Time B']), 'Gols Time B'] = ((((media_gols_fora_feitos+media_gols_casa_tomados)/2)*(1+(((20-posicao_fora)*2))/100)*fator_grandeza)*fator5jogos).round(0)

futuros_jogos.to_excel('jogos_futuros.xlsx')

In [ ]:
import pandas as pd
import numpy as np

df = df.dropna(subset=['Gols Time A'])
df = pd.concat([df, futuros_jogos], ignore_index=True)
classificacoes_por_rodada = []
for rodada in range(1, df['Rodada'].max()+1):
  df_rodada = df[df['Rodada'] <= rodada]
  classificacao = pd.DataFrame()
  classificacao['Posição'] = 0
  classificacao['Times'] = df['Time A'].unique()
  classificacao['Pontos'] = 0
  classificacao['Jogos'] = 0
  classificacao['Jogos Casa'] = 0
  classificacao['Jogos Fora'] = 0
  classificacao['Vitorias'] = 0
  classificacao['Vitorias Casa'] = 0
  classificacao['Vitorias Fora'] = 0
  classificacao['Derrotas'] = 0
  classificacao['Derrotas Casa'] = 0
  classificacao['Derrotas Fora'] = 0
  classificacao['Empates'] = 0
  classificacao['Empates Casa'] = 0
  classificacao['Empates Fora'] = 0
  classificacao['Gols Feitos'] = 0
  classificacao['Gols Feitos Casa'] = 0
  classificacao['Gols Feitos Fora'] = 0
  classificacao['Gols Tomados'] = 0
  classificacao['Gols Tomados Casa'] = 0
  classificacao['Gols Tomados Fora'] = 0
  classificacao['Saldo de Gols'] = 0
  classificacao['Saldo de Gols Casa'] = 0
  classificacao['Saldo de Gols Fora'] = 0
  classificacao['Aproveitamento'] = 0
  classificacao['Aproveitamento Casa'] = 0
  classificacao['Aproveitamento Fora'] = 0
  for index, row in df_rodada.iterrows():
    if row['Gols Time A'] > row['Gols Time B']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Vitorias Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Derrotas Fora'] += 1
    elif row['Gols Time B'] > row['Gols Time A']:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Derrotas Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Vitorias Fora'] += 1
    else:
      classificacao.loc[(classificacao['Times'] == row['Time A']), 'Empates Casa'] += 1
      classificacao.loc[(classificacao['Times'] == row['Time B']), 'Empates Fora'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Feitos Casa'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Feitos Fora'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Gols Tomados Casa'] += row['Gols Time B']
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Gols Tomados Fora'] += row['Gols Time A']
    classificacao.loc[(classificacao['Times'] == row['Time A']), 'Jogos Casa'] += 1
    classificacao.loc[(classificacao['Times'] == row['Time B']), 'Jogos Fora'] += 1
  principais =['Jogos','Vitorias', 'Derrotas', 'Empates','Jogos', 'Gols Feitos', 'Gols Tomados']
  for i in principais:
    classificacao[i] = classificacao[f'{i} Casa'] + classificacao[f'{i} Fora']
  #classificacao['Jogos'] = classificacao['Jogos Casa'] + classificacao['Jogos Fora']
  #classificacao['Vitorias'] = classificacao['Vitorias Casa'] + classificacao['Vitorias Fora']
  classificacao['Saldo de Gols Casa'] = classificacao['Gols Feitos Casa'] - classificacao['Gols Tomados Casa']
  classificacao['Saldo de Gols Fora'] = classificacao['Gols Feitos Fora'] - classificacao['Gols Tomados Fora']
  classificacao['Pontos'] = classificacao['Vitorias'] * 3 + classificacao['Empates'] * 1
  classificacao['Aproveitamento'] = classificacao['Pontos']/(classificacao['Jogos']*3)
  classificacao['Aproveitamento Casa'] = (classificacao['Vitorias Casa']*3 +classificacao['Empates Casa']*1)/(classificacao['Jogos Casa']*3)
  classificacao['Aproveitamento Fora'] = (classificacao['Vitorias Fora']*3 +classificacao['Empates Fora']*1)/(classificacao['Jogos Fora']*3)
  classificacao['Saldo de Gols'] = classificacao['Gols Feitos'] - classificacao['Gols Tomados']
  classificacao = classificacao.sort_values(by=['Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
  classificacao['Posição'] = list(range(1, 21))
  classificacao['Rodada'] = rodada
  classificacoes_por_rodada.append(classificacao)
classificacoes_df = pd.concat(classificacoes_por_rodada)
classificacoes_df = classificacoes_df.sort_values(by=['Rodada', 'Pontos', 'Vitorias', 'Saldo de Gols', 'Gols Feitos'], ascending=False)
classificacoes_df.to_excel('tabela_brasileiraocompleta.xlsx')
classificacoes_df

,Posição,Times,Pontos,Jogos,Jogos Casa,Jogos Fora,Vitorias,Vitorias Casa,Vitorias Fora,Derrotas,...,Gols Tomados,Gols Tomados Casa,Gols Tomados Fora,Saldo de Gols,Saldo de Gols Casa,Saldo de Gols Fora,Aproveitamento,Aproveitamento Casa,Aproveitamento Fora,Rodada
16,1,Palmeiras,78,38,19,19,23,15,8,6,...,35,15,20,37,27,10,0.684211,0.807018,0.561404,38
18,2,Botafogo,78,38,19,19,23,14,9,6,...,39,16,23,29,21,8,0.684211,0.789474,0.578947,38
14,3,Fortaleza,68,38,19,19,19,14,5,8,...,44,12,32,9,20,-11,0.596491,0.824561,0.368421,38
17,4,Flamengo,65,38,19,19,18,11,7,9,...,50,21,29,15,10,5,0.570175,0.666667,0.473684,38
0,5,Inter,65,38,19,19,17,12,5,7,...,41,19,22,17,15,2,0.570175,0.719298,0.421053,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,16,AtléticoGO,0,1,1,0,0,0,0,1,...,2,2,0,-1,-1,0,0.000000,0.000000,NaN,1
10,17,Bahia,0,1,0,1,0,0,0,1,...,2,0,2,-1,0,-1,0.000000,NaN,0.000000,1
11,18,Grêmio,0,1,0,1,0,0,0,1,...,2,0,2,-1,0,-1,0.000000,NaN,0.000000,1
9,19,Vitória,0,1,1,0,0,0,0,1,...,1,1,0,-1,-1,0,0.000000,0.000000,NaN,1


In [ ]:
enviar_para_mysql(classificacoes_df, 'tabela_minhabaseprevisao')

Dados inseridos com sucesso na tabela tabela_minhabaseprevisao
